In [1]:
import numpy as np
from Bresenheim import *
import cpm
import matplotlib.pyplot as plt
from scipy import ndimage
%gui qt

In [2]:
dimension = 256
cube = test(dimension,plotly=False,mayavi=False)

percentage of volume occupied by frc 18.239086866378784


In [3]:
print(np.sum(cube))
print(256**3)

3060011.0
16777216


In [4]:
# ### SET UP CPM ###
# # params : 
# dimension = 256
# number_of_types = 3
# temperature = 7
# # emmpty cube for now : 
# # cube = np.zeros((dimension,dimension,dimension))
# # cube_with_type = np.copy(cube.astype(np.uint32))
# # cube_with_type[cube==1] += 2**24
# # initialize : 
# simulation = cpm.Cpm(dimension, number_of_types, temperature)
# simulation.set_constraints(cell_type = 1,other_cell_type = 2,adhesion = 5*10**5)
# simulation.set_constraints(cell_type = 2,target_area = 15000, lambda_area=5*10**5)
# simulation.set_constraints(cell_type = 2,other_cell_type = 2,adhesion = 10**6)
# simulation.set_constraints(cell_type = 2,other_cell_type = 0,adhesion = 5*10**5)
# simulation.set_constraints(cell_type = 2, lambda_perimeter = 2, target_perimeter = 15)
# #simulation.initialize_from_array(cube_with_type,1)
# simulation.set_constraints(cell_type = 2, lambda_act = 30, max_act = 40) # 400,20

In [5]:
### SET UP CPM ###
# params from Inge: multiplicated the adhesion engergies by 10
# and because lambda of .1 not possible here. 
dimension = 256
number_of_types = 3
temperature = 7

# initialize : 
simulation = cpm.Cpm(dimension, number_of_types, temperature)
# LAmbdas ; 
simulation.set_constraints(cell_type = 2,target_area = 1800, lambda_area=250)
simulation.set_constraints(cell_type = 2, lambda_perimeter = 10, target_perimeter = 8600)
simulation.set_constraints(cell_type = 2, lambda_act = 2500, max_act = 42) # 160,40
# adhesion ; 
simulation.set_constraints(cell_type = 1,other_cell_type = 2,adhesion = 150)
simulation.set_constraints(cell_type = 2,other_cell_type = 2,adhesion = 150)
simulation.set_constraints(cell_type = 2,other_cell_type = 0,adhesion = 50)
# FRC :
# cube_with_type = np.copy(cube.astype(np.uint32))
# cube_with_type[cube==1] += 2**24
simulation.initialize_from_array(cube,1)


In [6]:
s = simulation.get_state()
def print_stuf(s):
    cellids = s % 2**24
    celltypes = s // 2**24
    n_free = np.sum(celltypes == 0)
    frc = np.sum(celltypes == 1)
    t = np.sum(celltypes == 2)
    print('Totpix: ',256**3)
    print()
    print('ECM   : ' ,n_free)
    print('FRC   : ' ,frc)
    print('TCell : ', t)
    print()
    # ids : 
    ids = np.unique(cellids)
    print('Cell IDs: ')
    print()
    for i_d in ids:
        print(i_d,' : ',np.sum(cellids == i_d))

    print(len(ids) + 1,' : ',np.sum(cellids == len(ids) + 1))
# print(len(np.where(celltypes == 0)[0]))
# print(len(np.where(cellids == 0)[0]))
print_stuf(s)

Totpix:  16777216

ECM   :  13717205
FRC   :  3060011
TCell :  0

Cell IDs: 

0  :  16777216
2  :  0


In [7]:
# # calc perimeter approx
r = (150 / ((4*np.pi)/3))**(1/3)
print(r)
print('frc', (4/3) * np.pi*10**3)
print(4*np.pi*r**2)

3.2961038252544337
frc 4188.790204786391
136.5248256296245


In [8]:
# some checks : 

simulation.run(10)


In [9]:
s = simulation.get_state()
print_stuf(s)

Totpix:  16777216

ECM   :  13717205
FRC   :  3060011
TCell :  0

Cell IDs: 

0  :  16777216
2  :  0


In [10]:
# add a cell : 
simulation.add_cell(10,128,128,2)


In [11]:
simulation.run(20)


In [12]:
#s = simulation.get_state()
print_stuf(s)

Totpix:  16777216

ECM   :  13715403
FRC   :  3060011
TCell :  1802

Cell IDs: 

0  :  16775414
2  :  1802
3  :  0


In [ ]:
# # plot in mayavi 3d slice : 
# mlab.clf()
# mlab.volume_slice(s)
# mlab.show()

In [ ]:
celltypes = s // 2**24
frc = np.sum(celltypes == 1)
t = np.sum(celltypes == 2)
mlab.clf()
mlab.contour3d(t)
mlab.show()

In [ ]:
#ndimage.measurements.center_of_mass(celltypes == 1)
cellids = s % 2**24
celltypes = s // 2**24
print(len(np.where(celltypes == 2)[0]))
print(len(np.where(cellids == 1)[0]))

In [ ]:
# plt.figure(figsize=(10,10))
# plt.imshow(celltypes[64])
# plt.show()
256**3

In [ ]:
simulation.run(20)
cellids = s % 2**24
celltypes = s // 2**24
print(len(np.where(celltypes == 2)[0]))

In [ ]:
#show_state(s)

In [ ]:
# save path of single cell over couple of simulations : 
# cell needs to be able to move : 
path = np.zeros((dimension,dimension,dimension))
act_state = simulation.get_act_state()
cell_state = simulation.get_state()
cofmass = []
# simulate 100 steps
for i in range(100):
    simulation.run(10)
    path = path + act_state
    cofmass.append(ndimage.measurements.center_of_mass(cell_state))
#show_state(path)

In [ ]:
# show_state(path)

In [ ]:
from mpl_toolkits import mplot3d
%matplotlib notebook
# plot path of center of mass  :
x = [c[0] for c in cofmass]
y = [c[1] for c in cofmass]
z = [c[2] for c in cofmass]

fig = plt.figure()
ax = plt.axes(projection='3d')
ax.plot3D(x,y,z)
ax.scatter3D(x[0],y[0],z[0],label = 'start')
ax.scatter3D(x[-1],y[-1],z[-1],label = 'end')
ax.legend()

In [ ]:
x = simulation.get_act_state()
figure = plt.figure(figsize = (15,15))
plt.imshow(x[128])
s = simulation.get_state()
# figure = plt.figure(figsize = (15,15))
# plt.imshow(s[128])=0)]

In [ ]:
print(x.shape)

In [ ]:
len(np.where(path != 0.)[0])/256**3

In [ ]:
# TRY placing cell at boundary ; 

dimension = 256
number_of_types = 3
temperature = 7

# initialize : 
simulation = cpm.Cpm(dimension, number_of_types, temperature)
# FRC :
# cube_with_type = np.copy(cube.astype(np.uint32))
# cube_with_type[cube==1] += 2**24
# LAmbdas ; 
simulation.set_constraints(cell_type = 2,target_area = 1800, lambda_area=250)
simulation.set_constraints(cell_type = 2, lambda_perimeter = 10, target_perimeter = 8600)
simulation.set_constraints(cell_type = 2, lambda_act = 100, max_act = 35) # 160,40
# adhesion ; 
simulation.set_constraints(cell_type = 1,other_cell_type = 2,adhesion = 150)
simulation.set_constraints(cell_type = 2,other_cell_type = 2,adhesion = 150)
simulation.set_constraints(cell_type = 2,other_cell_type = 0,adhesion = 50)
# simulation.initialize_from_array(cube_with_type,1)

simulation.add_cell(128,128,128,2)
s = simulation.get_state()
cellids = s % 2**24
celltypes = s // 2**24
print(len(np.where(celltypes == 2)[0]))
cell = celltypes[np.where(celltypes == 2)]
c_of_masses = ndimage.measurements.center_of_mass(s)
print(c_of_masses)

In [ ]:
simulation.run(1)

In [ ]:
def real_cofmass(cell):
    """ cell = simulation.get_state() % 2**24 == id """
    labels, num_features = ndimage.label(cell)
    if num_features == 1:
        return ndimage.measurements.center_of_mass(cell)
        print('1 feature')
    else:
        # take biggest blob :
        sizes = [np.count_nonzero(labels == i) for i in range(1,num_features + 1)]
        print(sizes)
        max_indeces = np.where(sizes == np.amax(sizes))[0]
        index = np.random.choice(max_indeces,1,replace = False)
        print(index)
        # set smaller patches to zero : 
        labels[labels != index+1] = 0
        print(np.unique(labels))
        print(labels.shape)
#         print(num_features,' features')
        return ndimage.measurements.center_of_mass(labels)
    
    
#     elif num_fetures =< 8
        
#     else:# num_features > 8:
#         # cell has some blobs not connected :
#         # find biggest 8 blobs : 
#         sizes = [np.count_nonzero(labels == i) for i in range(1,num_features + 1)]
#         max_indeces = np.where(sizes == max(sizes))
#         indeces = np.random.sample(max_indeces,8,replace = False)
#         labels = labels[indeces]
    
    
cell = simulation.get_state() % 2**24 == 1
c = real_cofmass(cell)
print(c)

In [ ]:
# run and visualizd : 
for i in range(10):
    simulation.run(1)
#     s = simulation.get_state()
#     cellids = s % 2**24
#     celltypes = s // 2**24
    print(len(np.where(cellids == 1)[0]))
    cell = simulation.get_state() % 2**24 == 1
    c_of_masses = real_cofmass(cell)
    print(c_of_masses)
    plt.imshow(s[int(c_of_masses[0])])
    plt.show()